In [13]:
trait Ord {
    // this cmp that < 0 iff this < that
    // this cmp that > 0 iff this > that
    // this cmp that == 0 iff this == that
    def cmp(that: Ord): Int
    def ===(that: Ord): Boolean = (this.cmp(that)) == 0
    def < (that: Ord): Boolean = (this cmp that) < 0
    def > (that: Ord): Boolean = (this cmp that) > 0
    def <= (that: Ord): Boolean = (this cmp that) <= 0
    def >= (that: Ord): Boolean = (this cmp that) >= 0
}
def max3(a: Ord, b: Ord, c: Ord) : Ord =
    if (a <= b) { if (b <= c) c else b }
    else        { if (a <= c) c else a }

defined trait Ord
defined function max3

In [13]:
// how to use max3?
max3(10, 9, 20)

cmd13.sc:1: type mismatch;
 found   : Int(10)
 required: cmd13.this.cmd12.Ord
val res13 = max3(10, 9, 20)
                 ^cmd13.sc:1: type mismatch;
 found   : Int(9)
 required: cmd13.this.cmd12.Ord
val res13 = max3(10, 9, 20)
                     ^cmd13.sc:1: type mismatch;
 found   : Int(20)
 required: cmd13.this.cmd12.Ord
val res13 = max3(10, 9, 20)
                        ^Compilation Failed

: 

In [14]:
case class OInt(val value: Int) extends Ord {
    def cmp(that: Ord): Int = {
        that match {
            case OInt(v) => value - v
        }
    }
}
case class OString(val value: String) extends Ord {
    def cmp(that: Ord): Int = {
        that match {
            case OString(v) => value.compare(v)
        }
    }
}

max3(OInt(3), OInt(2), OInt(10))
max3(OString("abc"), OString("aaa"), OString("a"))

defined class OInt
defined class OString
res13_2: Ord = OInt(10)
res13_3: Ord = OString("abc")

### Make `Ord` to `Ord[A]`

In [17]:
trait Ord[A] {
    def cmp(that: Ord[A]): Int
    def ===(that: Ord[A]): Boolean = (this.cmp(that)) == 0
    def < (that: Ord[A]): Boolean = (this cmp that) < 0
    def > (that: Ord[A]): Boolean = (this cmp that) > 0
    def <= (that: Ord[A]): Boolean = (this cmp that) <= 0
    def >= (that: Ord[A]): Boolean = (this cmp that) >= 0
    def value: A
}

def max3[A](a: Ord[A], b: Ord[A], c: Ord[A]): Ord[A] =
    if (a <= b) { if (b <= c) c else b }
    else        { if (a <= c) c else a }

defined trait Ord
defined function max3

In [18]:
case class OInt(val value: Int) extends Ord[Int] {
    def cmp(that: Ord[Int]): Int = value - that.value
}
case class OString(val value: String) extends Ord[String] {
    def cmp(that: Ord[String]): Int = value.compare(that.value)
}
max3(OInt(3), OInt(2), OInt(10)).value
max3(OString("abc"), OString("aaa"), OString("a")).value

defined class OInt
defined class OString
res17_2: Int = 10
res17_3: String = "abc"

In [19]:
case class OInt2(val value: Int) extends Ord[Int] {
    def cmp(that: Ord[Int]): Int = that.value - value
}
max3(OInt(1), OInt2(2), OInt(10)).value
// have to implement comparison with type A

defined class OInt2
res18_1: Int = 2

### Not to use `def value`

In [20]:
trait Ord[A] {
    def cmp(that: A): Int
    def ===(that: A): Boolean = (this.cmp(that)) == 0
    def < (that: A): Boolean = (this cmp that) < 0
    def > (that: A): Boolean = (this cmp that) > 0
    def <= (that: A): Boolean = (this cmp that) <= 0
    def >= (that: A): Boolean = (this cmp that) >= 0
}

def max3[A <: Ord[A]](a: A, b: A, c: A): A =
    if (a <= b) { if (b <= c) c else b }
    else        { if (a <= c) c else a }

defined trait Ord
defined function max3

In [21]:
// OInt is a subtype of Ord[OInt]
case class OInt(val value: Int) extends Ord[OInt] {
    def cmp(that: OInt): Int = value - that.value
}
case class OString(val value: String) extends Ord[OString] {
    def cmp(that: OString): Int = value.compare(that.value)
}
max3(OInt(3), OInt(2), OInt(10))
max3(OString("abc"), OString("aaa"), OString("a"))

defined class OInt
defined class OString
res20_2: OInt = OInt(10)
res20_3: OString = OString("abc")

In [23]:
case class OInt2(val value: Int) extends Ord[OInt] {
    def cmp(that: OInt): Int = that.value - value
}

defined class OInt2
defined class OInt3

In [21]:
max3(OInt(1), OInt2(2), OInt(10))

cmd21.sc:4: inferred type arguments [Product with cmd21.this.cmd19.Ord[cmd21.this.cmd20.OInt] with java.io.Serializable] do not conform to method max3's type parameter bounds [A <: cmd21.this.cmd19.Ord[A]]
val res21_1 = max3(OInt(1), OInt2(2), OInt(10))
              ^cmd21.sc:4: type mismatch;
 found   : cmd21.this.cmd20.OInt
 required: A
val res21_1 = max3(OInt(1), OInt2(2), OInt(10))
                       ^cmd21.sc:4: type mismatch;
 found   : Helper.this.OInt2
 required: A
val res21_1 = max3(OInt(1), OInt2(2), OInt(10))
                                 ^cmd21.sc:4: type mismatch;
 found   : cmd21.this.cmd20.OInt
 required: A
val res21_1 = max3(OInt(1), OInt2(2), OInt(10))
                                          ^Compilation Failed

: 

In [25]:
class OInt3(override val value: Int) extends OInt(value) {
    override def cmp(that: OInt): Int = that.value - value
}
max3(OInt(1), new OInt3(2), OInt(10))

defined class OInt3
res24_1: OInt = OInt(2)

In [22]:
class OInt2(val value2: Int) extends OInt(value2) {
    override def cmp(that: OInt): Int = that.value - value2
}
max3(OInt(1), new OInt2(2), OInt(10))

defined class OInt2
res21_1: OInt = OInt(2)

### Ordered Bag

In [26]:
class Bag[U <: Ord[U]] protected (val toList: List[U]) {
    def this() = this(Nil)
    def add(x: U) : Bag[U] = {
        def go(elmts: List[U]): List[U] =
            elmts match {
            case Nil => x :: Nil
            case e :: _ if (x < e) => x :: elmts
            case e :: _ if (x === e) => elmts
            case e :: rest => e :: go(rest)
        }
        new Bag(go(toList))
    }
}

defined class Bag

In [30]:
val emp = new Bag[OInt]()
val b = emp.add(OInt(3)).add(OInt(2)).add(OInt(10)).add(OInt(2))
b.toList.map((x) => x.value)
b.toList

emp: Bag[OInt] = ammonite.$sess.cmd25$Helper$Bag@11ef640f
b: Bag[OInt] = ammonite.$sess.cmd25$Helper$Bag@3d08ce8b
res29_2: List[Int] = List(2, 3, 10)
res29_3: List[OInt] = List(OInt(2), OInt(3), OInt(10))

In [32]:
val emp = new Bag[OString]()
val b = emp.add(OString("ABc")).add(OString("Abc")).add(OString("AbE"))
b.toList

emp: Bag[OString] = ammonite.$sess.cmd25$Helper$Bag@29990249
b: Bag[OString] = ammonite.$sess.cmd25$Helper$Bag@35f9516e
res31_2: List[OString] = List(OString("ABc"), OString("AbE"), OString("Abc"))

### OOP cannot make
* only supports elimination without introduction
* Interface was later developed than OOP

In [33]:
trait ListIF[L, A] {
    def empty: L
    def head(l: L): Option[A]
    def tail(l: L)
    def cons(a: A, l: L): L
    def append(l1: L, l2: L): L
}

defined trait ListIF

In [35]:
trait Ord[A] {
    // smallest elemnt in Ord[A]
    def smallestElmt: Ord[A]
}

// o.smallestElmt

defined trait Ord